In [1]:
!pip install selenium

In [2]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [12]:
""" NOTE: STILL A WIP :) """

import requests
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
import re

import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options) # for headless mode
# driver = webdriver.Chrome(ChromeDriverManager().install()) # if you want to see the browser

url = 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25240&siteid=5341#home'

driver.get(url)


"""ALL ANGULARJS –– looked into PhantomJS, etc., lots of dynamic but possibly different from Selenium uses"""

with requests.Session() as s: 
    r = s.get(url)
    src = driver.page_source #NOW HAVE ACCESS TO FULLY LOADED HTML *AND* JAVASCRIPT SOUP
    soup = bs(src)
    soup.prettify()
#     print(soup)
       
    pageUrl = []     # √ 
    jobPosition = [] # √
    location = []     
    description = []  
    requirements = [] 
    Immunization = [] 
    Economics = []    
    
    allLinks = []
    allJobNames = []
    allLocations = []
    
    data = soup.find_all('a', class_='jobProperty')
#     print("DATA: ")
#     print(data)

    for nameLink in data:
        link = nameLink['href']
        allLinks.append(link)
#         print(link)

        name = nameLink.text
        allJobNames.append(name)
#         print(name)

        
    jobIndex = 0
    for page in allLinks:
        result = s.get(page)
        driver.get(page); # v1 got too messy, v2 changed to Selenium rendering of HTML instead of BS4
        pageSource = driver.page_source
        soup = bs(pageSource) 
        soup.prettify()
        
        print(soup)
        # lots of research: not many resources on webscraping AngularJS using BS4 OR Selenium
        # soup is printing lots of json –– webcrawler extracts through json(?)
        # .. webcrawler may be the better and faster soln ..
        
        for script in soup(['script','style']):
            script.decompose()
        strips = list(soup.stripped_strings)
        strips = str(strips)
        

        data = soup.find("input")["value"]
        dataList = data.split("},{")
        relevantList = []
#         print("dataList: ")
#         print(dataList)
        descriptionString = ""
        for info in dataList:
#             print("INFO: ")
#             print(info)
            if "Job Summary" in info:
                descriptionString = "Job Summary\n" + info
                relevantList.append(info)
            descriptionString.replace(""""QuestionName":"Job Summary","AnswerValue":""""", "")
            print(descriptionString)
#         print("RELEVANTLIST: ")
#         print(relevantList)
                
        """PROBLEM: not extracting from traditional HTML using 
        traditional HTML tags, soup shows all data in 
        ONE LARGE paragraph filled with AngularJS –– 
        is resulting in way too much hardcoding"""

        immunization = ['Immunization', 'immunisation', 'vaccine', 'vaccines','vaccine-preventable diseases', 'vpd outbreak',
            'immunization campaign', 'SIA','supplemental immunization activities', 'cold chain', 'GAVI','shigella', 'cholera',
            'bcg', 'dtp', 'dpt', 'measles', 'influenza', 'conjugate vaccine']

        economics = ['Economics','expenditure tracking', 'financing', 
            'value for vaccination' , 'costing', 'economic analysis','costs' , 'equity', 'cost effectiveness', 'cost-effectiveness', 
            'cost benefit analysis', 'benefit-cost analysis','cost utility analysis','budget impact analysis' , 'budget' , 'budgeting' , 
            'GAVI','funding gap','fiscal']

        immResult = any(ele in strips for ele in immunization)
        ecResult = any(ele in strips for ele in economics)
        
        print(immResult, ecResult)
        
        if immResult or ecResult:
            if immResult: Immunization.append('True')
            else: Immunization.append('False')
            if ecResult: Economics.append('True')
            else: Economics.append('False')
            
           
            description.append(descriptionString)
            requirements.append(requirementsString)
            
            pageUrl.append(page)
            jobPosition.append(allJobNames[jobIndex])
            location.append(allLocations[jobIndex])
            
        jobIndex += 1
        

DataFrame = pd.DataFrame() 
DataFrame['Page Url']= pageUrl 
DataFrame['Job']= jobPosition 
DataFrame['Location'] = location
DataFrame['Description'] = description
DataFrame['Requirements'] = requirements
DataFrame['Immunization'] = Immunization
DataFrame['Economics'] = Economics

Data = DataFrame.drop_duplicates() 
Data.to_csv("Harvard_Data.csv")

driver.quit()

print('Webscraping complete')  

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/hur712/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 



































































Job Summary
"QuestionName":"Job Summary","AnswerValue":"Plan and conduct comprehensive applications/web development for complex projects; typically work as part of a team to implement complex business solutions. Perform expert coding: design, develop code at an expert level, and manage development projects, teams, programs. May manage multiple projects simultaneously.","VerityZone":"formtext32","QuestionType":"textarea","ActualValueFromSolar":null
Job Summary
"QuestionName":"Job Summary","AnswerValue":"Plan and conduct comprehensive applications/web development for complex projects; typically work as part of a team to implement complex business solutions. Perform expert coding: design, develop code at an expert level, and manage development projects, teams, programs. May manage multiple projects simultaneously.","VerityZone":"formtext32","QuestionType":"textarea","ActualValueFromSolar":null
Job Summary
"QuestionName":

NameError: name 'requirementsString' is not defined